In [ ]:
pip install d20

In [64]:
import pandas as pd
import random
from d20 import roll

In [65]:
excel_file = 'https://github.com/btaylor77/dnd/blob/main/beyond_pandas.xlsx?raw=true'
#excel_file = '/Users/Brad/Documents/git/dnd/beyond_pandas.xlsx'
hoard_df = pd.read_excel(excel_file,'hoard_items',engine='openpyxl').fillna(0)
spell_df = pd.read_excel(excel_file,'spell_list',index_col='Spell Name',engine='openpyxl')
class_df = pd.read_excel(excel_file,'class_spell_list',index_col='Spell Name',engine='openpyxl')
items_df = pd.read_excel(excel_file,'magic_items',index_col='item_name',engine='openpyxl')
coins_df = pd.read_excel(excel_file,'coins',engine='openpyxl').fillna(0)
treasure_df = pd.read_excel(excel_file,'treasure',engine='openpyxl').fillna(0)
damage_types = ['acid','cold','fire','force','lightning','necrotic','psychic','poison','radiant','thunder']
monster_types = ['Abberation','Beast','Celestial','Construct','Dragon','Elemental','Fey','Fiend','Giant','Humanoid','Monstrosity','Ooze','Plant','Undead']
swords = ['Greatsword','Longsword','Rapier','Scimitar','Shortsword']
weapons = swords + ['Battleaxe','Blowgun','Club','Crossbow, hand','Crossbow, heavy','Crossbow, light','Dagger','Dart','Flail','Glaive','Greataxe','Greatclub','Halberd','Handaxe','Javelin','Lance','Light hammer','Longbow','Mace','Maul','Morningstar','Net','Pike','Quarterstaff','Shortbow','Sickle','Sling','Spear','Trident','War pick','Warhammer','Whip',]

party = [('bard','base'),('bard','optional'),
          ('cleric','peace'),('cleric','base'),('cleric','optional'),
          ('warlock','base'),('warlock','fiend'),('warlock','optional'),
          ('wizard','base'),('wizard','optional')
        ]
all_class = [('artificer','base'),('bard','base'),('cleric','base'),('druid','base'),('paladin','base'),('ranger','base'),('sorcerer','base'),('wizard','base'),('warlock','base')]
arcane_class = [('artificer','base'),('bard','base'),('sorcerer','base'),('wizard','base'),('warlock','base')]
divine_class = [('cleric','base'),('druid','base'),('paladin','base'),('ranger','base')]


In [66]:
def roll_spells(min_level,max_level,classes,n=1):
    target_df = class_df.copy()
    target_df = target_df[(target_df['Level'] >= min_level) & (target_df['Level'] <= max_level) ]
    target_df = target_df[target_df[["Class","Subclass"]].apply(tuple, 1).isin(classes)]
    spells = target_df.index.values.tolist()
    select = pd.Series(random.choices(spells,k=n)).unique()
    target_df = spell_df.loc[select].sort_values(['Level','Spell Name'])
    return target_df

def roll_magic_items(table,n):
    if n > 0:
        target_df = items_df.copy()
        target_df = target_df[target_df['table_name']==table]
        target_df = target_df.reset_index()
        try:
            select = random.choices(target_df.index.values.tolist(),k=n,weights=list(target_df['weight']))
        except:
            print(table)
        target_df = target_df.loc[select]
        target_df['item_name'] = target_df.apply(lambda row: 'Scroll Of ' + roll_spells(row['spell_level'],row['spell_level'],party,n=1).index.values[0] if row['spell_level'] in range(9) else row['item_name'], axis=1)
        target_df['item_name'] = target_df.apply(lambda row: row['item_name'] + ' (' + random.choice(damage_types) + ')' if row['damage_type_flag'] == 1 else row['item_name'], axis=1)
        target_df['item_name'] = target_df.apply(lambda row: row['item_name'] + ' (' + random.choice(monster_types) + ')' if row['monster_type_flag'] == 1 else row['item_name'], axis=1)
        target_df['item_name'] = target_df.apply(lambda row: row['item_name'] + ' (' + random.choice(swords) + ')' if row['sword_flag'] == 1 else row['item_name'], axis=1)
        target_df['item_name'] = target_df.apply(lambda row: row['item_name'] + ' (' + random.choice(weapons) + ')' if row['weapon_flag'] == 1 else row['item_name'], axis=1)
        target_df = target_df.sort_values('item_name')
    else:
        target_df = pd.DataFrame(columns=['item_name'])
    return target_df

def roll_coins(crs=[],coin_type='single'):
    result_df = pd.DataFrame()
    for cr in crs:
        target_df = coins_df.copy()
        target_df = target_df[(target_df['min_cr']<= cr) & (target_df['max_cr'] >= cr) & (target_df['coin_type']==coin_type)]
        select = random.choices(target_df.index.values.tolist(),k=1,weights=list(target_df['weight']))
        target_df = target_df.loc[select][['CP','SP','EP','GP','PP']]
        result_df = result_df.append(target_df)
        result_df['CP'] = result_df['CP'].apply(lambda x: roll(str(x)).total)
        result_df['SP'] = result_df['SP'].apply(lambda x: roll(str(x)).total)
        result_df['EP'] = result_df['EP'].apply(lambda x: roll(str(x)).total)
        result_df['GP'] = result_df['GP'].apply(lambda x: roll(str(x)).total)
        result_df['PP'] = result_df['PP'].apply(lambda x: roll(str(x)).total)
        total_coins = pd.DataFrame(result_df.sum(axis=0)).transpose()
#        result_df = result_df.apply(lambda x: roll(x).total)
    result_df = total_coins.transpose().reset_index()
    result_df['treasure'] = result_df.apply(lambda x: str(x[0]) + ' ' + x['index'].lower(),axis=1)
    coins = result_df[result_df[0] > 0]['treasure'].values.tolist()
    return coins

def roll_treasure(gp,n):
    results = []
    if n > 0:
        target_df = treasure_df[treasure_df['GP']==gp]
        results = []
        for i in range(n):
            select = random.choice(target_df.index.values.tolist())
            results.append(select)
        target_df = target_df.loc[results].groupby('treasure_name').count()
        target_df = target_df.rename(columns={"GP":"count"})
        target_df['each'] = target_df.apply(lambda row: ' each' if row['count']>1 else '',axis=1)

        target_df = target_df.reset_index()
        target_df['treasure_desc'] = target_df.apply(lambda row: str(row['count']) + ' ' + row['treasure_name'] + ' worth ' + str(gp) + 'gp' + row['each'],axis=1)
        results = target_df['treasure_desc'].values.tolist()
    return results

def roll_hoard(cr):
    loot = []
    target_df = hoard_df.copy()
    target_df['cr'] = cr
    target_df = target_df[(target_df['min_cr']<= cr) & (target_df['max_cr'] >= cr)]
    select = random.choices(target_df.index.values.tolist(),k=1,weights=list(target_df['weight']))
    target_df = target_df.loc[select]
    target_df['treasure_die'] = target_df['treasure_die'].apply(lambda x: roll(str(x)).total)
    target_df['magic_item_die_1'] = target_df['magic_item_die_1'].apply(lambda x: roll(str(x)).total)
    target_df['magic_item_die_2'] = target_df['magic_item_die_2'].apply(lambda x: roll(str(x)).total)
    target_df = target_df.drop(columns=['min_cr','max_cr','weight'])
    items1 = list(roll_magic_items(target_df['magic_item_table_1'].values[0],target_df['magic_item_die_1'].values[0])['item_name'].values.tolist())
    items2 = list(roll_magic_items(target_df['magic_item_table_2'].values[0],target_df['magic_item_die_2'].values[0])['item_name'].values.tolist())
    treasure = roll_treasure(target_df['treasure_value'].values[0],target_df['treasure_die'].values[0])
    coins = roll_coins([cr],'hoard')
    loot = loot + treasure + coins + items1 + items2
    target_df = target_df.reset_index(drop=True)

    return loot#target_df

In [67]:
for i in roll_hoard(12):
    print(i)

1 Blue Sapphire worth 1000gp
1 Emerald worth 1000gp
2 Fire Opal worth 1000gp each
1 Opal worth 1000gp
2 Star Ruby worth 1000gp each
2 Star Sapphire worth 1000gp each
1 Yellow Sapphire worth 1000gp
8000 gp
900 pp
All-Purpose Tool, +1
Figurine of Wondrous Power (Bronze Griffon)
Gem of Seeing


In [68]:
for i in roll_coins([3,3,3,3,3,2,2],'single'):
    print(i)

40 sp
15 ep
28 gp


In [82]:
player_sb = ["Acid Splash","Animate Objects","Banishment","Bigby's Hand","Blight","Color Spray",
             "Comprehend Languages","Cone of Cold","Continual Flame","Counterspell","Detect Magic",
             "Dispel Magic","Earth Tremor","Earthbind","Far Step","Find Familiar","Fly","Fog Cloud",
             "Greater Invisibility","Ice Knife","Identify","Leomund’s Tiny Hut","Lightning Bolt",
             "Mage Armor","Mage Hand","Magic Missile","Mind Sliver","Mirror Image","Misty Step",
             "Poison Spray","Polymorph","Ray of Frost","Shield","Sleep","Snilloc’s Snowball Swarm",
             "Steel Wind Strike","Stoneskin","Summon Fey","Tasha's Mind Whip","Vitriolic Sphere",
             "Water Breathing"]

school_weight = {
    'abjuration':2,
    'conjuration':1,
    'divination':1,
    'enchantment':1,
    'evocation':1,
    'illusion':1,
    'necromancy':2,
    'transmutation':1,
}
spellbook = class_df[class_df['Class']=='wizard'].copy()
spellbook['weight'] = spellbook.apply(lambda x: school_weight[x['School']], axis=1)
spellbook.loc[spellbook.index.isin(player_sb), 'weight']= 0.5

In [83]:
def generate_spellbook(min_level=1,max_level=9,n=6,weight={}):
    select = []
    df = spellbook.copy()
    for i in range(min_level,max_level+1):
        if n > 0:
            if i < max_level+1:
                x = max(1,random.randrange(0,(n+1)//2))
            else:
                x = n
            lvl_df = df[df['Level']==i]
            lvl_select = random.choices(lvl_df.index.values.tolist(),k=x,weights=list(lvl_df['weight']))
            select = select + lvl_select
            n = n - x
    df = df.loc[pd.Series(select).unique()].sort_values(['Level','School'])
    return df

def display_spellbook(df):
    for level in df['Level'].unique():
        print(f'Level {level} Spells:' + '(' + str(len(df[df['Level']==level])) + ')')
        print(", ".join(map(str,df[df['Level']==level].index.values)))
        print('')

In [91]:
sb = generate_spellbook(1,6,25,school_weight)

display_spellbook(sb)
len(sb)

Level 1 Spells:(3)
Earth Tremor, False Life, Jump

Level 2 Spells:(1)
Rope Trick

Level 3 Spells:(1)
Magic Circle

Level 4 Spells:(9)
Private Sanctum, Evard's Black Tentacles, Conjure Minor Elementals, Summon Greater Demon, Dimension Door, Summon Construct, Charm Monster, Sickening Radiance, Vitriolic Sphere

Level 5 Spells:(3)
Contact Other Plane, Geas, Wall of Force

Level 6 Spells:(1)
Guards And Wards



18

In [58]:
t1 = class_df[class_df['Class']=='wizard'].copy()
t1['weight'] = t1.apply(lambda x: school_weight[x['School']], axis=1)
t1[t1.index.isin(player_sb)]['weight'] = 0.25


/var/folders/ts/p_br4n612d7g0h1bcszsdl0c0000gn/T/ipykernel_74236/929758954.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1[t1.index.isin(player_sb)]['weight'] = 0.25


In [59]:
t1.loc[t1.index.isin(player_sb), 'weight']= 0.25

,Class,Subclass,Level,School,weight
Spell Name,,,,,
Color Spray,wizard,base,1,illusion,0.25
Comprehend Languages,wizard,base,1,divination,0.25
Detect Magic,wizard,base,1,divination,0.25
Earth Tremor,wizard,base,1,evocation,0.25
Find Familiar,wizard,base,1,conjuration,0.25
Fog Cloud,wizard,base,1,conjuration,0.25
Ice Knife,wizard,base,1,conjuration,0.25
Identify,wizard,base,1,divination,0.25
Mage Armor,wizard,base,1,abjuration,0.25
